## Import Data

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense
from keras.models import Sequential

In [4]:
## import the data
DATADIR = ".//Data//solid_classification"
CATEGORIES = ["Cardboard", "Glass","Metal","Paper","Plastic"]
for category in CATEGORIES:
# create path to cars and trucks
    path = os.path.join(DATADIR,category)  
# iterate over each image per cars and trucks
    for img in os.listdir(path):
# convert to array
        img_array = cv2.imread(os.path.join(path,img))

In [5]:
## create training data
IMG_SIZE = 80
training_data = []

In [6]:
## create traing data
def create_training_data():
    for category in CATEGORIES:  # do cars and trucks

        path = os.path.join(DATADIR,category)  # create path to cars and trucks
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=solid 1=wet

        for img in tqdm(os.listdir(path)):  # iterate over each image cars and trucks
            try:
        # convert to array
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)  
                # resize to normalize data size
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass

create_training_data()

100%|██████████████████████████████████████████████████████████████████████████████| 999/999 [00:00<00:00, 1209.95it/s]


In [8]:
## create features and labels
import random
random.shuffle(training_data)

X_solid = []
y_solid = []

for features,label in training_data:
    X_solid.append(features)
    y_solid.append(label)

X_solid = np.array(X_solid).reshape(-1,
                                    IMG_SIZE, IMG_SIZE,1)

In [10]:
##write the pickle file
import pickle

pickle_out = open("X_solid.pickle","wb")
pickle.dump(X_solid, pickle_out)
pickle_out.close()

pickle_out = open("y_solid.pickle","wb")
pickle.dump(y_solid, pickle_out)
pickle_out.close()

In [11]:
##divide training and testing dataset
import pickle
import numpy

# -------------Load X--------------
pickle_in = open("X_solid.pickle","rb") 
train_images = pickle.load(pickle_in)
# Split part of X for testing purpose
test_images = train_images[int(0.9 * len(train_images)):]
train_images = train_images[:int(0.9 * len(train_images))]

# -------------Load y--------------
pickle_in = open("y_solid.pickle","rb") 
train_labels = pickle.load(pickle_in)
#Split part of Y for testing purpose
test_labels = train_labels[int(0.9 * len(train_labels)):]
train_labels = numpy.asarray(train_labels[:int(0.9 * len(train_labels))])

# convert list to ndarray, and reshape the labels
test_labels = numpy.array(test_labels).reshape(len(test_labels),1)
train_labels = numpy.array(train_labels).reshape(len(train_labels),1)

In [12]:
print(f"train_images - Type: {type(train_images)}, Shape {train_images.shape}")
print(f"test_images - Type: {type(test_images)}, Shape {test_images.shape}")

train_images - Type: <class 'numpy.ndarray'>, Shape (4496, 80, 80, 1)
test_images - Type: <class 'numpy.ndarray'>, Shape (500, 80, 80, 1)


In [13]:
train_images

array([[[[206],
         [205],
         [204],
         ...,
         [149],
         [149],
         [147]],

        [[204],
         [205],
         [204],
         ...,
         [147],
         [147],
         [146]],

        [[203],
         [203],
         [204],
         ...,
         [145],
         [145],
         [144]],

        ...,

        [[198],
         [198],
         [197],
         ...,
         [173],
         [175],
         [174]],

        [[200],
         [199],
         [197],
         ...,
         [175],
         [177],
         [176]],

        [[200],
         [199],
         [196],
         ...,
         [176],
         [178],
         [177]]],


       [[[210],
         [208],
         [210],
         ...,
         [158],
         [149],
         [152]],

        [[208],
         [210],
         [203],
         ...,
         [151],
         [153],
         [154]],

        [[208],
         [204],
         [203],
         ...,
         [153],
         [

##  Build model

In [14]:
## import package
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [15]:
## set the randomseed
from DL_tools import fixRandomSeed as frs
frs.fixRandomSeed()

fixRandomSeed in action


In [16]:
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [19]:
model_solid = Sequential([
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(train_images.shape[1:])),
    MaxPooling2D(pool_size=2),

    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),

    Flatten(),

    Dense(64, activation='relu'),

    Dense(5, activation='softmax')
])

In [20]:
##Compile the model
model_solid.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [21]:
history = model_solid.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
141/141 [==============================] - 25s 169ms/step - loss: 1.5485 - accuracy: 0.2627 - val_loss: 1.3786 - val_accuracy: 0.4420
Epoch 2/10
141/141 [==============================] - 25s 174ms/step - loss: 1.3226 - accuracy: 0.4641 - val_loss: 1.2587 - val_accuracy: 0.4980
Epoch 3/10
141/141 [==============================] - 24s 173ms/step - loss: 1.2056 - accuracy: 0.5181 - val_loss: 1.1673 - val_accuracy: 0.5080
Epoch 4/10
141/141 [==============================] - 25s 174ms/step - loss: 1.1040 - accuracy: 0.5503 - val_loss: 1.0993 - val_accuracy: 0.5600
Epoch 5/10
141/141 [==============================] - 24s 172ms/step - loss: 0.9692 - accuracy: 0.6237 - val_loss: 0.9510 - val_accuracy: 0.6200
Epoch 6/10
141/141 [==============================] - 25s 177ms/step - loss: 0.8685 - accuracy: 0.6657 - val_loss: 1.0717 - val_accuracy: 0.5900
Epoch 7/10
141/141 [==============================] - 24s 173ms/step - loss: 0.7934 - accuracy: 0.6932 - val_loss: 0.9273 - val_ac

In [22]:
# Evaluate the model
test_loss, test_acc = model_solid.evaluate(test_images,  test_labels, verbose=2)
print(f"Loss = {round(test_loss,2)}, Acc = {round(test_acc,2)}")

16/16 - 1s - loss: 0.9160 - accuracy: 0.6460
Loss = 0.92, Acc = 0.65


## Test data

In [61]:
import cv2

#打开摄像头
cap = cv2.VideoCapture(0)

while(1):
    # 读取图片
    ret, frame = cap.read()
    # 展示获取的图片
    cv2.imshow("capture", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # 图片写入文件
        cv2.imwrite("photo.jpg", frame)
        break

cap.release()
cv2.destroyAllWindows()

In [65]:
photo = cv2.imread("photo.jpg", cv2.IMREAD_GRAYSCALE)
new_photo = cv2.resize(photo, (IMG_SIZE, IMG_SIZE))
new_photo = new_photo.reshape(-1,IMG_SIZE, IMG_SIZE, 1)
new_photo = new_photo / 255.0

In [76]:
print("Predict result is:",CATEGORIES[model_solid.predict_classes(new_photo)[0]])

Predict result is: Glass


array([[0.00134905, 0.54679245, 0.28230035, 0.0059488 , 0.16360942]],
      dtype=float32)

In [59]:
train_images.shape[1:]

(80, 80, 1)